# Problem 3

In [ ]:
import PIL
import glob
import math
from random import uniform
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

## Functions

In [ ]:
def rotate(img, angle_deg):
    angle_rad = angle_deg * math.pi / 180
    rotate_img = img.rotate(angle_deg, PIL.Image.BILINEAR)
    sin, cos = abs(math.sin(angle_rad)), abs(math.cos(angle_rad))
    crop = 64 * (cos - sin)/(cos**2 - sin**2)
    upper = left = math.ceil((64-crop)/2)
    lower = right = math.ceil((64-crop)/2 + crop)
    rotate_img = rotate_img.crop(box=(left, upper, right, lower))
    rotate_img = rotate_img.resize((64, 64), PIL.Image.BILINEAR)
    return rotate_img

In [ ]:
def scale(img, factor):
    crop = 64 * (1 - factor / 100)
    upper = left = math.ceil((64-crop)/2)
    lower = right = math.ceil((64-crop)/2 + crop)
    scale_img = img.crop(box=(left, upper, right, lower))
    scale_img = scale_img.resize((64, 64), PIL.Image.BILINEAR)
    return scale_img

In [ ]:
def load_train(path,
               augmentation=False,
               augmentation_size=10,
               max_rotate=30,
               max_scale=50,
               verbose=False):
    x, y = [], []
    for file in glob.glob(path + "**/*.jpg", recursive=True):
        img = Image.open(file)
        try:
            data = list(tuple(p) for p in img.getdata())
        except:
            if verbose:
                print("Error loading file {}".format(file))
        else:
            x.append(np.array(data).reshape(64, 64, 3))
            y.append(0 if "Cat" in file else 1)
            if augmentation:
                for _ in range(augmentation_size):
                    #scale
                    scale_img = scale(img, uniform(1, max_scale))
                    data = list(tuple(p) for p in scale_img.getdata())
                    x.append(np.array(data).reshape(64, 64, 3))
                    y.append(0 if "Cat" in file else 1)
                    #rotate
                    rotate_img = rotate(img, uniform(-max_scale, max_scale))
                    data = list(tuple(p) for p in scale_img.getdata())
                    x.append(np.array(data).reshape(64, 64, 3))
                    y.append(0 if "Cat" in file else 1)
    x = np.asarray(x).swapaxes(1, 3).swapaxes(2, 3).astype("float32") / 255.
    y = np.asarray(y).reshape(-1, 1)
    return x, y

Note it is not the best way to call *augmentation_size* times the function *scale* and *rotate* (see [here](https://wiki.python.org/moin/PythonSpeed/PerformanceTips)), but it is easier to understand.

## Model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2, 2), nn.ReLU(),
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2, 2), nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=5, stride=1), nn.MaxPool2d(2, 2),
            nn.ReLU())

        self.mlp = nn.Sequential(
            nn.Linear(32 * 6 * 6, 100), nn.ReLU(), nn.Linear(100, 1))

    def forward(self, x):
        x = self.conv(x)
        x = torch.sigmoid(self.mlp(x.view(-1, 32 * 6 * 6)))
        return x

    def evaluate(self, loader):
        with torch.no_grad():
            correct, total = 0, 0
            for data in loader:
                # get the inputs
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                ouputs = self(inputs)
                val_loss = criterion(ouputs, labels)
                total += labels.size(0)
                predicted = (ouputs > 0.5).float()
                correct += (predicted == labels).sum().item()
#                 print("7")
        return val_loss, correct / total

## Load data

In [ ]:
x, y = load_train("data/kaggle/trainset/", augmentation=False, verbose=True)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(
    x, y, test_size=0.1, stratify=y)

In [ ]:
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).float()
x_valid = torch.from_numpy(x_valid).float()
y_valid = torch.from_numpy(y_valid).float()

In [ ]:
train = torch.utils.data.TensorDataset(x_train, y_train)
trainloader = torch.utils.data.DataLoader(train, batch_size=1, shuffle=True)

In [ ]:
valid = torch.utils.data.TensorDataset(x_valid, y_valid)
validloader = torch.utils.data.DataLoader(valid, batch_size=1, shuffle=True)

## Train model

In [ ]:
model = Net()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Let\'s use {}".format(device))

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
best_val_acc, train_loss, val_loss = -1, None, None
print("{:5s} | {:10s} | {:10s} | {:10s}".format("epoch", "train_loss",
                                                "valid_loss", "valid_acc"))
for epoch in range(25):  # loop over the dataset multiple times
    model.train()
    for i, data in enumerate(trainloader, 1):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        train_loss = criterion(outputs, labels)
        train_loss.backward()
        optimizer.step()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model")
        print("\tsaved!", end="")
    print("")
model.load_state_dict(torch.load("best_model"))